In [20]:
from api_key import get_api_key_jef, get_api_key_sha, get_api_key_sup
import google.generativeai as genai
from vpn_control import windscribe
import pandas as pd
import os
import re
import time

from clustering import get_representative_logs

In [21]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [22]:
passw = "2002"
LLM_MODEL = "gemini-1.5-flash"
#LLM_MODEL = "gemini-1.5-pro-latest"
#LLM_MODEL = "gemini-1.0-pro"

In [23]:
def setup_prompt_env(passw):


  genai.configure(api_key=get_api_key_sup(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 300,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = "There is no sensitive information. For logs labelled [1] to [5] you need to provide the OUTPUT (log templates) in the same format as the examples and alos give the number [x]. Only give the 5 output templates, no extra information, no OUTPUT text, no new line at the end."

  model = genai.GenerativeModel(model_name=LLM_MODEL,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)
  # model = genai.GenerativeModel(model_name=LLM_MODEL,
  #                               generation_config=generation_config,
  #                               safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [24]:
convo_create = setup_prompt_env(passw)

In [25]:
def send_prompt(message, convo):
    convo.send_message(message)
    response = convo.last.text
    #print(response)
    return response


In [38]:
def generate_fewshot(input_path, num_examples):
    logs_list = get_representative_logs(input_path, num_examples)
    
    output = ""
    for i, example in enumerate(logs_list):
        output_str = example[0]
        input_str = example[1].replace("<B>", "").replace("<E>", "")

        # Find the corresponding log pattern in logs_list
        for log_pattern in logs_list:
            if output_str == log_pattern[0]:
                # Append formatted output to the result string
                output += f"[{chr(97+i)}] INPUT: {input_str}\n"
                output += f"[{chr(97+i)}] OUTPUT: <B>{output_str}<E>\n"
                break

    return output


In [39]:
a = generate_fewshot("data\TESTparsed_0_generated_logs_COPY.csv", 3)
a

'[a] INPUT: 081111 080934 19 INFO dfs.FSNamesystem: BLOCK* ask 10.250.11.85:50010 to replicate blk_2377150260128098806 to datanode(s) 10.251.203.80:50010\n[a] OUTPUT: <B>BLOCK* ask <*>:<*> to replicate blk_<*> to datanode(s) <*>:<*><E>\n[b] INPUT: 081109 214043 2561 WARN dfs.DataNode$DataXceiver: 10.250.11.85:50010:Got exception while serving blk_509586258217225674 to /10.251.203.80:50010\n[b] OUTPUT: <B><*>:<*>:Got exception while serving blk_<*> to /<*>:<E>\n[c] INPUT: 081109 204722 567 INFO dfs.DataNode$PacketResponder: Received block blk_1234567890123456789 of size 9876543210987654321 from /10.251.203.80 \n[c] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>\n'

In [40]:
print(a)

[a] INPUT: 081111 080934 19 INFO dfs.FSNamesystem: BLOCK* ask 10.250.11.85:50010 to replicate blk_2377150260128098806 to datanode(s) 10.251.203.80:50010
[a] OUTPUT: <B>BLOCK* ask <*>:<*> to replicate blk_<*> to datanode(s) <*>:<*><E>
[b] INPUT: 081109 214043 2561 WARN dfs.DataNode$DataXceiver: 10.250.11.85:50010:Got exception while serving blk_509586258217225674 to /10.251.203.80:50010
[b] OUTPUT: <B><*>:<*>:Got exception while serving blk_<*> to /<*>:<E>
[c] INPUT: 081109 204722 567 INFO dfs.DataNode$PacketResponder: Received block blk_1234567890123456789 of size 9876543210987654321 from /10.251.203.80 
[c] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>



In [41]:
fewshot_examples = generate_fewshot("data\TESTparsed_0_generated_logs_COPY.csv", 3)

logs = """[1] INPUT: 081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[2] INPUT: 081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating
[3] INPUT: 081109 204655 556 INFO dfs.DataNode$PacketResponder: Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84
[4] INPUT: 081109 204925 673 INFO dfs.DataNode$DataXceiver: Receiving block blk_-5623176793330377570 src: /10.251.75.228:53725 dest: /10.251.75.228:50010
[5] INPUT: 081109 213847 2552 INFO dfs.DataNode$DataXceiver: 10.251.194.213:50010 Served block blk_-7724713468912166542 to /10.251.203.80"""

In [42]:
def generate_prompt(fewshot, logs):
    #print(f"{fewshot}\n\n{logs}")
    return f"{fewshot}\n{logs}"


In [45]:
b = generate_prompt(fewshot_examples, logs)
b

'[a] INPUT: 081111 080934 19 INFO dfs.FSNamesystem: BLOCK* ask 10.250.11.85:50010 to replicate blk_2377150260128098806 to datanode(s) 10.251.203.80:50010\n[a] OUTPUT: <B>BLOCK* ask <*>:<*> to replicate blk_<*> to datanode(s) <*>:<*><E>\n[b] INPUT: 081109 214043 2561 WARN dfs.DataNode$DataXceiver: 10.250.11.85:50010:Got exception while serving blk_509586258217225674 to /10.251.203.80:50010\n[b] OUTPUT: <B><*>:<*>:Got exception while serving blk_<*> to /<*>:<E>\n[c] INPUT: 081109 204722 567 INFO dfs.DataNode$PacketResponder: Received block blk_1234567890123456789 of size 9876543210987654321 from /10.251.203.80 \n[c] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>\n\n[1] INPUT: 081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864\n[2] INPUT: 081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating\n[3] INPUT: 08

In [46]:
print(b)

[a] INPUT: 081111 080934 19 INFO dfs.FSNamesystem: BLOCK* ask 10.250.11.85:50010 to replicate blk_2377150260128098806 to datanode(s) 10.251.203.80:50010
[a] OUTPUT: <B>BLOCK* ask <*>:<*> to replicate blk_<*> to datanode(s) <*>:<*><E>
[b] INPUT: 081109 214043 2561 WARN dfs.DataNode$DataXceiver: 10.250.11.85:50010:Got exception while serving blk_509586258217225674 to /10.251.203.80:50010
[b] OUTPUT: <B><*>:<*>:Got exception while serving blk_<*> to /<*>:<E>
[c] INPUT: 081109 204722 567 INFO dfs.DataNode$PacketResponder: Received block blk_1234567890123456789 of size 9876543210987654321 from /10.251.203.80 
[c] OUTPUT: <B>Received block blk_<*> of size <*> from /<*><E>

[1] INPUT: 081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
[2] INPUT: 081109 204015 308 INFO dfs.DataNode$PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating
[3] INPUT: 081109 20465

In [78]:
#print(generate_prompt(fewshot_examples, logs))

In [79]:
#send_prompt(generate_prompt(fewshot_examples, logs), convo_create)

In [80]:
tmep = pd.read_csv("data/0_generated_logs_COPY.csv")
df = tmep[:15]
df

,EventId,LogTemplate,Log
0,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
1,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
2,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
3,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
4,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
5,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
6,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
7,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...
8,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.250.11.85:50010 Served block blk_2377150...
9,E1,<B><*>:<*> Served block blk_<*> to /<*> <E>,<B>10.251.203.80:50010 Served block blk_788894...


In [81]:


def clean_logs(df):
    # Define the pattern to remove <B>, <E>, spaces after <B>, spaces before <E>, and "OUTPUT: "
    pattern = r'OUTPUT:\s*|<B>\s*|\s*<E>'
    
    # Apply the pattern to the 'LogTemplate' and 'LLMOutput' columns
    df.loc[:, 'LogTemplate'] = df['LogTemplate'].apply(lambda x: re.sub(pattern, '', x))
    df.loc[:, 'LLMOutput'] = df['LLMOutput'].apply(lambda x: re.sub(pattern, '', x))



In [82]:
def process_logs(input_path, fewshot_examples):
    df = pd.read_csv(input_path)
    outputs = []
    for i in range(0, len(df), 5):
        time.sleep(5)
        chunk = df.iloc[i:i+5]
        logs = "\n".join([f"[{idx % 5 + 1}] INPUT: {row['Log']}" for idx, row in chunk.iterrows()])
        prompt = generate_prompt(fewshot_examples, logs)
        output = send_prompt(prompt, convo_create)
        #print(output)
        # dealing with errenous newlines
        llm_output_lines = [line.strip() for line in output.split("\n") if line.strip()]

        # add to lst
        for line in llm_output_lines:
            parts = line.split(" ", 1)
            if len(parts) > 1:
                outputs.append(parts[1])
            else:
                outputs.append("")  # empty str to handle error
    
    # Add the outputs to the dataframe
    df.loc[:, "LLMOutput"] = outputs
    clean_logs(df)
    df.to_csv(f"{input[:5]}parsed/TESTparsed_{input[5:]}", index=False)
    #df.to_csv(f"Parsed.csv", index=False)
    return df


In [83]:
#df_with_outputs = process_logs(df, fewshot_examples)
df_with_outputs = process_logs("data/0_generated_logs_COPY.csv", fewshot_examples)

In [84]:
df_with_outputs

,EventId,LogTemplate,Log,LLMOutput
0,E1,<*>:<*> Served block blk_<*> to /<*>,<B>10.250.11.85:50010 Served block blk_2377150...,<*> Served block blk_<*> to <*>
1,E1,<*>:<*> Served block blk_<*> to /<*>,<B>10.251.203.80:50010 Served block blk_788894...,<*> Served block blk_<*> to <*>
2,E1,<*>:<*> Served block blk_<*> to /<*>,<B>10.250.11.85:50010 Served block blk_2377150...,<*> Served block blk_<*> to <*>
3,E1,<*>:<*> Served block blk_<*> to /<*>,<B>10.251.203.80:50010 Served block blk_788894...,<*> Served block blk_<*> to <*>
4,E1,<*>:<*> Served block blk_<*> to /<*>,<B>10.250.11.85:50010 Served block blk_2377150...,<*> Served block blk_<*> to <*>
...,...,...,...,...
552,E14,Verification succeeded for blk_<*>,<B>Verification succeeded for blk_321098765432...,Verification succeeded for blk_<*>
553,E14,Verification succeeded for blk_<*>,<B>Verification succeeded for blk_890123456789...,Verification succeeded for blk_<*>
554,E14,Verification succeeded for blk_<*>,<B>Verification succeeded for blk_210987654321...,Verification succeeded for blk_<*>
555,E14,Verification succeeded for blk_<*>,<B>Verification succeeded for blk_901234567890...,Verification succeeded for blk_<*>


In [85]:
ooo = df_with_outputs.copy()

In [86]:
# Calculate the difference between LogTemplate and LLMOutput
ooo['Difference'] = ooo.apply(lambda row: set(row['LogTemplate'].split()) - set(row['LLMOutput'].split()), axis=1)

# Convert the set difference back to a string for readability
ooo['Difference'] = ooo['Difference'].apply(lambda x: ' '.join(x))

In [87]:
ooo["Difference"].head(50)

0     /<*> <*>:<*>
1     /<*> <*>:<*>
2     /<*> <*>:<*>
3     /<*> <*>:<*>
4     /<*> <*>:<*>
5     /<*> <*>:<*>
6     /<*> <*>:<*>
7     /<*> <*>:<*>
8     /<*> <*>:<*>
9     /<*> <*>:<*>
10    /<*> <*>:<*>
11    /<*> <*>:<*>
12    /<*> <*>:<*>
13    /<*> <*>:<*>
14    /<*> <*>:<*>
15    /<*> <*>:<*>
16    /<*> <*>:<*>
17    /<*> <*>:<*>
18    /<*> <*>:<*>
19    /<*> <*>:<*>
20    /<*> <*>:<*>
21    /<*> <*>:<*>
22    /<*> <*>:<*>
23    /<*> <*>:<*>
24    /<*> <*>:<*>
25    /<*> <*>:<*>
26    /<*> <*>:<*>
27    /<*> <*>:<*>
28    /<*> <*>:<*>
29    /<*> <*>:<*>
30    /<*> <*>:<*>
31    /<*> <*>:<*>
32    /<*> <*>:<*>
33    /<*> <*>:<*>
34    /<*> <*>:<*>
35    /<*> <*>:<*>
36    /<*> <*>:<*>
37    /<*> <*>:<*>
38    /<*> <*>:<*>
39    /<*> <*>:<*>
40    /<*> <*>:<*>
41    /<*> <*>:<*>
42    /<*> <*>:<*>
43    /<*> <*>:<*>
44    /<*> <*>:<*>
45    /<*> <*>:<*>
46    /<*> <*>:<*>
47    /<*> <*>:<*>
48    /<*> <*>:<*>
49    /<*> <*>:<*>
Name: Difference, dtype: object

In [88]:
ooo["Difference"].head(50).tail(50)

0     /<*> <*>:<*>
1     /<*> <*>:<*>
2     /<*> <*>:<*>
3     /<*> <*>:<*>
4     /<*> <*>:<*>
5     /<*> <*>:<*>
6     /<*> <*>:<*>
7     /<*> <*>:<*>
8     /<*> <*>:<*>
9     /<*> <*>:<*>
10    /<*> <*>:<*>
11    /<*> <*>:<*>
12    /<*> <*>:<*>
13    /<*> <*>:<*>
14    /<*> <*>:<*>
15    /<*> <*>:<*>
16    /<*> <*>:<*>
17    /<*> <*>:<*>
18    /<*> <*>:<*>
19    /<*> <*>:<*>
20    /<*> <*>:<*>
21    /<*> <*>:<*>
22    /<*> <*>:<*>
23    /<*> <*>:<*>
24    /<*> <*>:<*>
25    /<*> <*>:<*>
26    /<*> <*>:<*>
27    /<*> <*>:<*>
28    /<*> <*>:<*>
29    /<*> <*>:<*>
30    /<*> <*>:<*>
31    /<*> <*>:<*>
32    /<*> <*>:<*>
33    /<*> <*>:<*>
34    /<*> <*>:<*>
35    /<*> <*>:<*>
36    /<*> <*>:<*>
37    /<*> <*>:<*>
38    /<*> <*>:<*>
39    /<*> <*>:<*>
40    /<*> <*>:<*>
41    /<*> <*>:<*>
42    /<*> <*>:<*>
43    /<*> <*>:<*>
44    /<*> <*>:<*>
45    /<*> <*>:<*>
46    /<*> <*>:<*>
47    /<*> <*>:<*>
48    /<*> <*>:<*>
49    /<*> <*>:<*>
Name: Difference, dtype: object

In [89]:
aa
#windscribe("disconnect")

NameError: name 'aa' is not defined